In [1]:


!pip install cohere --quiet
!pip install -U llama-index llama-index-llms-cohere --quiet
!pip install llama-index-llms-gemini --quiet
!pip install bert-score
!pip install -U weaviate-client --quiet
!pip install -U llama-index --quiet
%pip install -U llama-index-embeddings-huggingface --quiet
!pip install -U llama-index-vector-stores-weaviate --quiet
!pip install -U bitsandbytes --quiet
!pip install -U llama-index-llms-huggingface --quiet
%pip install llama-index-retrievers-bm25 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 132.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 14.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently

In [48]:
from llama_index.core import Document
import torch
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.embeddings import BaseEmbedding
from sklearn.preprocessing import normalize
from llama_index.core import Document
import torch
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.embeddings import BaseEmbedding
from sklearn.preprocessing import normalize
# Embedding model2️⃣ AfriBERTa embedding (HuggingFace style)
# ============================
class AfriBERTaEmbedding(BaseEmbedding):
    def __init__(self, model_name="castorini/afriberta_base", **kwargs):
        super().__init__(**kwargs)
        self._device = "cuda" if torch.cuda.is_available() else "cpu"
        from transformers import AutoTokenizer, AutoModel
        self._tokenizer = AutoTokenizer.from_pretrained(model_name)
        self._model = AutoModel.from_pretrained(model_name).to(self._device)
        self._model.eval()

    def _mean_pooling(self, token_embeddings, attention_mask):
        mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * mask_expanded, 1)
        sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

    def _embed(self, texts):
        if isinstance(texts, str):
            texts = [texts]
        inputs = self._tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(self._device)

        with torch.no_grad():
            outputs = self._model(**inputs)
            embeddings = self._mean_pooling(outputs.last_hidden_state, inputs["attention_mask"])
            embeddings = normalize(embeddings.cpu().numpy(), norm="l2")
        return embeddings.tolist()

    # -----------------------------
    # Implement required abstract methods
    # -----------------------------
    def _get_text_embedding(self, text: str):
        return self._embed(text)[0]

    def _get_query_embedding(self, query: str):
        return self._embed(query)[0]

    async def _aget_text_embedding(self, text: str):
        return self._get_text_embedding(text)

    async def _aget_query_embedding(self, query: str):
        return self._get_query_embedding(query)


      # -----------------------------
    # ✅ Compatibility Alias
    # -----------------------------
    def encode(self, texts, **kwargs):
        """Alias to maintain compatibility with retrievers expecting .encode()"""
        return self._embed(texts)

from llama_index.core.settings import Settings
Settings.embed_model = AfriBERTaEmbedding()

from llama_index.core.settings import Settings
Settings.embed_model = AfriBERTaEmbedding()

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of XLMRobertaModel were not initialized from the model checkpoint at castorini/afriberta_base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLMRobertaModel were not initialized from the model checkpoint at castorini/afriberta_base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this mo

In [ ]:
import os

os.environ["WEAVIATE_URL"] = "https://eu3nsymbtcib2x8fy120yw.c0.europe-west3.gcp.weaviate.cloud"
os.environ["WEAVIATE_API_KEY"] = "**************************************** "
import os
import weaviate
from weaviate.classes.init import Auth

# Best practice: store your credentials in environment variables
weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]
# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

print(client.is_ready())

True


In [4]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import SimpleDirectoryReader

In [5]:

#3. Wrap your Weaviate collection
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name="YorubaChunk",   # use your actual collection name
)
from llama_index.core import VectorStoreIndex, StorageContext
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# 6️⃣ Create index with Qdrant
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    storage_context=storage_context,
    embed_model=Settings.embed_model
)


In [6]:
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
from collections import defaultdict

# Dense Retriever (AfriBERTa)
dense_retriever = index.as_retriever(similarity_top_k=5)

# Sparse Retriever (BM25)
yoruba_collection = client.collections.get("YorubaChunk")
all_docs = [Document(text=obj.properties["text"]) for obj in yoruba_collection.iterator()]
sparse_retriever = BM25Retriever(all_docs, similarity_top_k=5)

# Hybrid Retriever
class HybridRetriever(BaseRetriever):
    def __init__(self, dense_retriever, sparse_retriever, mode="rrf", alpha=0.5, k=60):
        self.dense_retriever = dense_retriever
        self.sparse_retriever = sparse_retriever
        self.mode = mode
        self.alpha = alpha
        self.k = k

    def _retrieve(self, query, **kwargs):
        dense_results = self.dense_retriever.retrieve(query, **kwargs)
        sparse_results = self.sparse_retriever.retrieve(query, **kwargs)

        dense_dict = {r.node.node_id: (r.score, i + 1) for i, r in enumerate(dense_results)}
        sparse_dict = {r.node.node_id: (r.score, i + 1) for i, r in enumerate(sparse_results)}

        all_doc_ids = set(dense_dict.keys()) | set(sparse_dict.keys())
        fused_scores = defaultdict(float)

        if self.mode == "rrf":
            for doc_id in all_doc_ids:
                if doc_id in dense_dict:
                    _, rank = dense_dict[doc_id]
                    fused_scores[doc_id] += 1.0 / (self.k + rank)
                if doc_id in sparse_dict:
                    _, rank = sparse_dict[doc_id]
                    fused_scores[doc_id] += 1.0 / (self.k + rank)
        elif self.mode == "rsf":
            for doc_id in all_doc_ids:
                s_vec = dense_dict.get(doc_id, (0.0, None))[0]
                s_bm25 = sparse_dict.get(doc_id, (0.0, None))[0]
                fused_scores[doc_id] = self.alpha * s_vec + (1 - self.alpha) * s_bm25

        fused_results = []
        for doc_id, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True):
            node = None
            for r in dense_results + sparse_results:
                if r.node.node_id == doc_id:
                    node = r.node
                    break
            fused_results.append(NodeWithScore(node=node, score=score))
        return fused_results

hybrid_retriever = HybridRetriever(dense_retriever, sparse_retriever, mode="rrf")


DEBUG:bm25s:Building index from IDs objects


In [7]:
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig
from llama_index.core import Settings
import torch
import cohere
from llama_index.llms.gemini import Gemini


# =============================
# 1. HuggingFace Model Loader
# =============================
def load_huggingface_llm(model_name: str):
    """
    Load a HuggingFace model using 4-bit quantization with BitsAndBytes.
    Automatically detects if model is seq2seq or causal.
    Falls back to standard precision if quantization fails.
    """
    from transformers import (
        AutoTokenizer,
        AutoModelForCausalLM,
        AutoModelForSeq2SeqLM,
        BitsAndBytesConfig,
    )
    import torch

    print(f"🔄 Loading {model_name}...")

    # ✅ Define quantization configuration (stable for CUDA 12+)
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",              # recommended quantization type
        bnb_4bit_compute_dtype=torch.bfloat16,  # use bfloat16 for CUDA 12+
        bnb_4bit_use_double_quant=True
    )

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

    # Detect model architecture
    if any(x in model_name.lower() for x in ["t5", "mbart", "aya", "afrolm", "bart"]):
        model_loader = AutoModelForSeq2SeqLM
    else:
        model_loader = AutoModelForCausalLM

    try:
        # Attempt 4-bit quantized loading
        model = model_loader.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=quant_config,
            torch_dtype=torch.bfloat16,
            trust_remote_code=True
        )
        print(f"✅ Successfully loaded {model_name} in 4-bit quantized mode.")
    except Exception as e:
        print(f"[WARN] 4-bit quantization failed for {model_name}: {e}")
        print("➡️ Falling back to full precision mode.")
        model = model_loader.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.float16 if torch.cuda.is_available() else "auto",
            trust_remote_code=True
        )

    # Wrap model in HuggingFaceLLM
    llm = HuggingFaceLLM(
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512
    )

    return llm

# =============================
# 2. Gemini Model Loader
# =============================
def load_gemini_llm(api_key: str, model: str = "models/gemini-2.5-flash"):
    """Load Google Gemini model."""
    print(f"🔄 Loading Gemini: {model}...")
    llm = Gemini(model=model, api_key=api_key)
    print(f"✅ Gemini loaded successfully!")
    return llm


# =============================
# 3. Cohere Setup
# =============================
def setup_cohere_client(api_key: str):
    """Initialize Cohere client."""
    print("🔄 Setting up Cohere client...")
    co = cohere.ClientV2(api_key=api_key)
    print("✅ Cohere client ready!")
    return co


# =============================
# 3.5 Document Cleaning Functions
# =============================
def clean_retrieved_text(text: str) -> str:
    """
    Clean and normalize retrieved text before passing to generation.
    """
    if not text or not isinstance(text, str):
        return ""

    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text)

    # Remove special characters that might confuse the model
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]', '', text)

    # Remove URLs (optional - comment out if URLs are important)
    # text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)

    # Remove email addresses (optional)
    # text = re.sub(r'\S+@\S+', '', text)

    # Remove excessive punctuation
    text = re.sub(r'([.!?])\1+', r'\1', text)

    # Trim whitespace
    text = text.strip()

    return text


def filter_and_clean_nodes(nodes, min_length: int = 50, max_length: int = 2000, score_threshold: float = 0.0):
    """
    Filter and clean retrieved nodes based on quality criteria.

    Args:
        nodes: Retrieved nodes from the retriever
        min_length: Minimum character length for a node to be kept
        max_length: Maximum character length (truncate if longer)
        score_threshold: Minimum relevance score (if available)

    Returns:
        List of dicts with cleaned text and score
    """
    cleaned_nodes = []

    for node in nodes:
        # Get the text content
        text = node.text if hasattr(node, 'text') else str(node)

        # Clean the text
        cleaned_text = clean_retrieved_text(text)

        # Skip if too short after cleaning
        if len(cleaned_text) < min_length:
            continue

        # Truncate if too long
        if len(cleaned_text) > max_length:
            cleaned_text = cleaned_text[:max_length] + "..."

        # Check relevance score if available
        score = getattr(node, 'score', None)
        if score is not None and score < score_threshold:
            continue

        # Store cleaned text and score in a dict
        cleaned_nodes.append({'text': cleaned_text, 'score': score})

    return cleaned_nodes


def deduplicate_contexts(nodes):
    """
    Remove duplicate or highly similar contexts.
    Accepts list of dicts with 'text' key.
    """
    seen_texts = set()
    unique_nodes = []

    for node in nodes:
        text = node['text']

        # Simple deduplication based on first 100 characters
        text_signature = text[:100].lower().strip()

        if text_signature not in seen_texts:
            seen_texts.add(text_signature)
            unique_nodes.append(node)

    return unique_nodes


def yoruba_rag_query_llamaindex(topic: str, top_k: int = 3, clean_context: bool = True):
    """
    RAG query using LlamaIndex LLM (HuggingFace or Gemini via Settings.llm).
    Now includes document cleaning before generation.
    """
    # Retrieve relevant documents
    retrieved_nodes = dense_retriever.retrieve(topic)[:top_k]
    print(f"📚 Retrieved {len(retrieved_nodes)} documents")

    # Clean and filter retrieved documents
    if clean_context:
        print("🧹 Cleaning retrieved documents...")
        retrieved_nodes = filter_and_clean_nodes(
            retrieved_nodes,
            min_length=50,
            max_length=2000,
            score_threshold=0.0
        )
        retrieved_nodes = deduplicate_contexts(retrieved_nodes)
        print(f"✨ After cleaning: {len(retrieved_nodes)} documents")

    # Concatenate contexts
    context = "\n\n".join([n['text'] for n in retrieved_nodes])
    print(f"📝 Context length: {len(context)} characters")

    # Apply Yoruba synthesis prompt
    full_prompt = prompt_answer.format(context=context)
    print("--- Prompt sent to LLM ---")
    print(full_prompt[:500] + "..." if len(full_prompt) > 500 else full_prompt)

    # Generation configuration for better quality responses
    generation_config = {
        "temperature": 0.7,             # Controls creativity
        "top_p": 0.9,                   # Nucleus sampling
        "top_k": 40,                    # Limits token search space
        "repetition_penalty": 1.15,     # Penalize repeating n-grams
        "max_new_tokens": 256,          # Limit response length
        "no_repeat_ngram_size": 3,      # Avoid loops
        "do_sample": True               # Enables random sampling
    }

    # Call LLM using Settings.llm with generation config
    # Detect Gemini model by class name
    llm_class_name = type(Settings.llm).__name__.lower()
    if "gemini" in llm_class_name:
        response = Settings.llm.complete(full_prompt)
    else:
        response = Settings.llm.complete(full_prompt, **generation_config)


    return response.text


def yoruba_rag_query_cohere(topic: str, cohere_client, top_k: int = 5, clean_context: bool = True):
    """
    RAG query using Cohere API.
    Now includes document cleaning before generation.
    """
    # Retrieve relevant documents
    retrieved_nodes = hybrid_retriever.retrieve(topic)[:top_k]
    print(f"📚 Retrieved {len(retrieved_nodes)} documents")

    # Clean and filter retrieved documents
    if clean_context:
        print("🧹 Cleaning retrieved documents...")
        retrieved_nodes = filter_and_clean_nodes(
            retrieved_nodes,
            min_length=50,
            max_length=2000,
            score_threshold=0.0
        )
        retrieved_nodes = deduplicate_contexts(retrieved_nodes)
        print(f"✨ After cleaning: {len(retrieved_nodes)} documents")

    # Concatenate contexts
    context = "\n\n".join([n['text'] for n in retrieved_nodes])

    # Call Cohere Chat API
    response = cohere_client.chat(
        model="command-a-03-2025",
        messages=[
            {
                "role": "user",
                "content": f"""
                Ìbéèrè: {topic}

                Ìtàn àkọ́kọ́ (context):
                {context}

                Ìdáhùn ní èdè Yorùbá:
                """
            }
        ]
    )

    return response.message.content[0].text

In [ ]:
from sentence_transformers import SentenceTransformer, util
# Load your embedding model
similarity_model=SentenceTransformer("BAAI/bge-m3")
from bert_score import score

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [8]:
!pip install rouge

In [9]:
from typing import List, Union
from sentence_transformers import SentenceTransformer, util
from bert_score import score
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Initialize models
similarity_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
rouge = Rouge()

def evaluate_generation_metrics(
    question: str,
    contexts: Union[str, List[str]],
    answer: str,
    reference: str
):
    """
    Evaluate RAG output quality using BLEU, ROUGE-L, BERTScore, and cosine similarity.
    """
    # Normalize inputs
    if isinstance(contexts, str):
        contexts = [contexts]
    full_context = ' '.join(contexts)

    # --- BLEU ---
    smoothie = SmoothingFunction().method4
    bleu_score = sentence_bleu(
        [reference.split()],
        answer.split(),
        smoothing_function=smoothie
    )

    # --- ROUGE-L ---
    rouge_scores = rouge.get_scores(answer, reference, avg=True)
    rouge_l = rouge_scores['rouge-l']['f']

    # --- BERTScore ---
    P, R, F1 = score(
        [answer],
        [reference],
        lang='yo',
        model_type='xlm-roberta-large'
    )
    bert_f1 = F1.mean().item()

    # --- Cosine Similarity (semantic similarity) ---
    ans_emb = similarity_model.encode(answer, convert_to_tensor=True)
    ref_emb = similarity_model.encode(reference, convert_to_tensor=True)
    cosine_sim = util.cos_sim(ans_emb, ref_emb).item()

    # --- Composite Generation Score ---
    composite_gen = round(
        (bleu_score + rouge_l + bert_f1 + cosine_sim) / 4, 3
    )

    return {
        'BLEU': round(bleu_score, 3),
        'ROUGE-L': round(rouge_l, 3),
        'BERTScore_F1': round(bert_f1, 3),
        'Cosine_Similarity': round(cosine_sim, 3),
        'Composite_Gen': composite_gen   # ✅ renamed for pipeline consistency
    }


def evaluate_yoruba_rag(question: str, contexts: Union[str, List[str]], answer: str, reference: str):
    """
    Combined RAG evaluation: context-level and generation-level.
    """
    # Context relevance
    CR = evaluate_context_relevance(question, contexts)
    # Faithfulness
    FG = evaluate_faithfulness(answer, contexts)
    # Generation-level metrics
    gen_scores = evaluate_generation_metrics(question, contexts, answer, reference)

    # Composite
    composite = (CR + FG + gen_scores['Composite_Generation_Score']) / 3

    return {
        'Context_Relevance': round(CR, 3),
        'Faithfulness': round(FG, 3),
        **gen_scores,
        'Composite_RAG_Score': round(composite, 3)
    }


# --- Supporting functions reused from earlier ---
def evaluate_context_relevance(arg1: Union[str, List[str]], arg2: Union[str, List[str]]):
    if isinstance(arg1, str) and not isinstance(arg2, str):
        question = arg1
        contexts = arg2
    elif isinstance(arg2, str) and not isinstance(arg1, str):
        question = arg2
        contexts = arg1
    else:
        question = arg1
        contexts = arg2

    if isinstance(contexts, str):
        contexts = [contexts]

    if len(contexts) == 0:
        return 0.0

    question_emb = similarity_model.encode(question, convert_to_tensor=True)
    doc_embs = similarity_model.encode(contexts, convert_to_tensor=True)
    sims = util.cos_sim(question_emb, doc_embs)
    return float(sims.mean().item())


def evaluate_faithfulness(answer: str, context: Union[str, List[str]]):
    if isinstance(context, str):
        context = [context]
    ctx_emb = similarity_model.encode(' '.join(context), convert_to_tensor=True)
    ans_emb = similarity_model.encode(answer, convert_to_tensor=True)
    cosine_sim = util.cos_sim(ctx_emb, ans_emb).item()
    return float(cosine_sim)

def evaluate_answer_relevance(question: str, answer: str):
    # bert-score expects lists of references & candidates
    P, R, F1 = score([answer], [question], lang='yo', model_type='xlm-roberta-large')
    return float(F1.mean().item())

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
from sklearn.cluster import KMeans
def cluster_sample(df, domain_col="domain", n_per_domain=5):
    """
    Cluster-based sampling: use embeddings of questions to select 10 diverse samples per domain.
    """
    embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    sampled = []

    for domain, group in df.groupby(domain_col):
        group = group.dropna(subset=["question"])
        if len(group) <= n_per_domain:
            sampled.append(group)
            continue

        embeddings = embedder.encode(group["question"].tolist())
        kmeans = KMeans(n_clusters=n_per_domain, random_state=42)
        clusters = kmeans.fit_predict(embeddings)
        group["cluster"] = clusters

        cluster_sampled = (
            group.groupby("cluster")
            .apply(lambda x: x.sample(1, random_state=42))
            .reset_index(drop=True)
        )
        sampled.append(cluster_sampled)

    return pd.concat(sampled, ignore_index=True)

In [11]:
# Suppose you already have your context documents (from ground_truth.csv)
import pandas as pd
df = pd.read_csv("/content/ground_truth.csv")
sampled_df = cluster_sample(df, n_per_domain=2)



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-3234888018.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(1, random_state=42))
/tmp/ipython-input-3234888018.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(1, random_state=42))
/tmp/ipython-input-3234888018.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in 

In [ ]:
 # ====================================
# ⚡ Optimized Yoruba RAG Quantitative Evaluation Across 7 Models
# ====================================
import gc
import re
import time
from tqdm import tqdm
from functools import lru_cache
import pandas as pd
import torch


# ====================================
# 🧹 Utility Functions
# ====================================
def clean_text(text, max_len=512):
    """Normalize Yoruba text and truncate to avoid long context embedding."""
    text = re.sub(r'\s+', ' ', text.strip())
    return text[:max_len]


def free_memory():
    """Force garbage collection and clear GPU memory."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def get_free_gpu_memory():
    """Return free GPU memory (in GB) for device 0."""
    if not torch.cuda.is_available():
        return 0
    gpu_stats = torch.cuda.mem_get_info()
    free_mem_gb = gpu_stats[0] / 1024**3
    return round(free_mem_gb, 2)

# ====================================
# ⚙️ Model Setup (Cached)
# ====================================
@lru_cache(maxsize=8)
def set_model(model_name):
    """Load or cache model/LLM client."""
    from llama_index.core import Settings
    free_mem = get_free_gpu_memory()
    print(f"🧠 Detected free GPU memory: {free_mem} GB")

    if model_name == "gemini":
        llm = load_gemini_llm(api_key="*******************************")
        Settings.llm = llm
        return llm

    elif model_name == "cohere":
        global cohere_client
        cohere_client = setup_cohere_client(api_key="******************************")
        return None

    else:
        llm = load_huggingface_llm(model_name)
        Settings.llm = llm
        return llm


# ====================================
# 📊 Evaluation Configuration
# ====================================
model_names = [
    "gemini" , # Gemini API
    #"cohere",  # Cohere API
"mistralai/Mistral-7B-Instruct-v0.3",
    #"meta-llama/Llama-3.2-1B-Instruct",
    #"Qwen/Qwen2.5-7B-Instruct",
    #"bigscience/bloomz-7b1",
    #"deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"


]

from huggingface_hub import login
# Paste your HuggingFace access token here
login("hf_token_api_key")



In [59]:
modular_prompt = """
# ADVANCED YORÙBÁ KNOWLEDGE RETRIEVAL AND GENERATION

## CONTEXT DOCUMENT (ÀKỌ́LẸ́ ÌSÀLẸ̀ ÌMỌ̀)
Wàá lo àkọsílẹ̀ yìí nìkan láti dáhùn ìbéèrè tó tẹ̀lé e. Má ṣe lo ìmọ̀ tó kọjá èyí tó wà nínú àkọsílẹ̀ yìí.
---
{context}
---

## USER'S QUERY (ÌBÉÈRÈ OLÙLÒ)
Dáhùn ìbéèrè tí ó wà nísàlẹ̀ yìí dáradára, láti inú àkọsílẹ̀ tó wà lókè.
---
{query}
---

## YORÙBÁ OUTPUT REQUIREMENTS (ÀWỌN ÒFIN ÌDÁHÙN YORÙBÁ)
1. **Èdè:** Gbogbo ìdáhùn gbọ́dọ̀ wà ní **Èdè Yorùbá** pẹ̀lú **àmì-ohùn** tí ó tọ́ (ọ, ṣ, ẹ, à, è, ì, ò, ù).
2. **Ìdáhùn:** Ìdáhùn gbọ́dọ̀ jẹ́ **kíkún àti wíwọ́kẹ́rẹ́kẹ́rẹ́**, ní fífẹṣẹ̀ rẹ nínú àkọsílẹ̀.
3. **Ìgbàwọ̀n:** Ìdáhùn gbọ́dọ̀ jẹ́ **3 sí 5 gbólóhùn** ní gígùn.
4. **Ìgbékalẹ̀:** Bẹ̀rẹ̀ ìdáhùn rẹ ní tààràtà.

## ANSWER (ÌDÁHÙN)
"""


In [70]:
# ============================================================
# 🔹 MODULES
# ============================================================
import collections # Added import for OrderedDict if needed, but not strictly for this fix
from llama_index.core.schema import NodeWithScore # Explicitly import NodeWithScore for clarity
from typing import List, Dict, Any, Optional, Callable
import time
import random
import logging
import torch

logger = logging.getLogger(__name__)
# The original call_llm was a RAG function, let's redefine it here to be a simple LLM call
def _simple_llm_complete(llm_instance, prompt: str) -> str:
    return llm_instance.complete(prompt).text.strip()


class PreRetrievalModule:
    """Expands query using HyDE (Hypothetical Document Embedding)."""
    def __init__(self, llm: Optional[Any] = None, enable_hyde: bool = True, enable_expansion: bool = True):
        self.llm = llm
        self.enable_hyde = enable_hyde
        self.enable_expansion = enable_expansion

    def process(self, query: str) -> List[str]:
        """Apply HyDE and query expansion to produce query variants."""
        queries = [query]

        if self.enable_hyde and self.llm:
            hyde_prompt = f"Write a short factual paragraph that could answer: '{query}'"
            hypo_doc = _simple_llm_complete(self.llm, hyde_prompt)
            if hypo_doc:
                queries.append(hypo_doc)

        if self.enable_expansion and self.llm:
            expansion_prompt = f"Expand this question with related terms: '{query}'"
            expansion = _simple_llm_complete(self.llm, expansion_prompt)
            if expansion:
                queries.append(expansion)

        return queries


class RetrievalModule:
    """Combines dense and sparse retrieval (hybrid)."""
    def __init__(self, dense_retriever: Any, sparse_retriever: Optional[Any] = None, hybrid_alpha: float = 0.5):
        self.dense = dense_retriever
        self.sparse = sparse_retriever
        self.hybrid_alpha = hybrid_alpha

    def retrieve(self, queries: List[str], top_k: int = 10) -> List[NodeWithScore]: # Adjusted return type hint
        all_results_raw = []  # Store raw NodeWithScore objects

        for q in queries:
            # Dense retrieval
            dense_results = self.dense.retrieve(q)
            all_results_raw.extend(dense_results)

            if self.sparse:
                # Sparse retrieval
                sparse_results = self.sparse.retrieve(q)
                all_results_raw.extend(sparse_results)

        # Use a dictionary to deduplicate based on node_id, preserving NodeWithScore
        unique_nodes_map = collections.OrderedDict()
        for node_with_score in all_results_raw:
            # Assuming node_with_score.node.node_id is unique enough
            unique_nodes_map[node_with_score.node.node_id] = node_with_score

        # Return list of NodeWithScore objects, limited by top_k
        return list(unique_nodes_map.values())[:top_k]


class PostRetrievalModule:
    """Applies reranking, filtering, and compression."""
    def __init__(self, embed_model):
        self.embed_model = embed_model

    def rerank(self, nodes, query, top_k=3):
        if not nodes:
            return []
        # Use _embed and convert to tensor
        query_emb = torch.tensor(self.embed_model._embed(query)[0])
        node_texts = [n.text for n in nodes]
        node_embs = torch.tensor(self.embed_model._embed(node_texts))

        scores = torch.nn.functional.cosine_similarity(query_emb.unsqueeze(0), node_embs)
        ranked = sorted(zip(nodes, scores.tolist()), key=lambda x: x[1], reverse=True)
        return [n for n, _ in ranked[:top_k]]

    def context_filter(self, nodes, diversity_threshold=0.8):
        unique_nodes, seen = [], []
        for n in nodes:
            text = n.text.strip()
            if not text:
                continue
            # Use _embed and convert to tensor
            emb = torch.tensor(self.embed_model._embed(text[:512])[0])
            if all(torch.nn.functional.cosine_similarity(emb, s, dim=0) < diversity_threshold for s in seen):
                unique_nodes.append(n)
                seen.append(emb)
        return unique_nodes

    def compress_contexts(self, nodes, max_len=1500):
        texts = []
        total_len = 0
        for n in nodes:
            t = n.text.strip()
            if total_len + len(t) > max_len:
                break
            texts.append(t)
            total_len += len(t)
        return "\n\n".join(texts)


class GenerationModule:
    """Handles prompt construction and self-verification."""
    def __init__(self, llm, verify=True):
        self.llm = llm
        self.verify = verify

    def generate(self, query, context):
        prompt = modular_prompt.format(query=query, context="\n".join(context))
        response = self.llm.complete(prompt).text.strip()
        if self.verify:
            verif_prompt = f"""
            Verify if this Yoruba answer is consistent with the question:
            Question: {query}
            Answer: {response}
            Respond with 'Bẹẹni' (Yes) or 'Rárá' (No).
            """
            check = _simple_llm_complete(self.llm, verif_prompt).lower()
            if "rárá" in check:
                response += "\n\n[⚠️ Ìtẹ́wọ̀gbà: Ìdáhùn lè ní àìtọ́ díẹ̀]"
        return response


class OrchestrationModule:
    """Coordinates all plug-in modules."""
    def __init__(self, pre, retriever, post, generator):
        self.pre = pre
        self.retriever = retriever
        self.post = post
        self.generator = generator

    def modular_query(self, question: str, top_k: int = 5):
        # Step 1: Expand
        expanded = self.pre.process(question)
        print(f"🔍 Expanded Queries: {expanded}")

        # Step 2: Retrieve
        nodes = self.retriever.retrieve(expanded, top_k=top_k)
        print(f"📚 Retrieved {len(nodes)} documents")

        # Step 3: Post-Retrieval Filtering
        nodes = self.post.rerank(nodes, question, top_k=top_k)
        nodes = self.post.context_filter(nodes)
        context = self.post.compress_contexts(nodes)

        # Step 4: Generate
        answer = self.generator.generate(question, context)

        # Step 5: Verification
        # The generate method handles internal verification based on self.generator.verify
        verified = self.generator.verify

        return {
            "question": question,
            "answer": answer,
            "verified": verified,
            "context": context,
            "num_docs": len(nodes),
            "expanded_queries": expanded
        }

In [71]:
llm=set_model( "gemini")
pre_module = PreRetrievalModule(llm)
retr_module = RetrievalModule(dense_retriever, sparse_retriever)
post_module = PostRetrievalModule(Settings.embed_model)
gen_module = GenerationModule(llm)
modular_rag = OrchestrationModule(pre_module, retr_module, post_module, gen_module)

In [75]:
question=sampled_df["question"][1]
question

'Kí ni ìtúmọ̀ àṣà wọ́ *ṣòkòtò àti agbádá* nínú àṣà Yorùbá, àti kí ni ìtẹ́lọ́run tó wà nínú lílo wọn?'

In [76]:
response = modular_rag.modular_query(question=question,top_k=5)
answer = response["answer"]

🔍 Expanded Queries: ['Kí ni ìtúmọ̀ àṣà wọ́ *ṣòkòtò àti agbádá* nínú àṣà Yorùbá, àti kí ni ìtẹ́lọ́run tó wà nínú lílo wọn?', 'Wíwọ *ṣòkòtò àti agbádá* jẹ́ àmì pàtàkì nínú àṣà Yorùbá, tí ó dúró fún ọlá, ipò, àti ìwọ̀ntúnwọ̀nsì. Kì í ṣe aṣọ lásán, bí kò ṣe ìfihàn ìdánimọ̀ Yorùbá àti ìsopọ̀ pẹ̀lú àṣà àti ìtàn. Wọ́n máa ń wọ̀ wọ́n ní àwọn ayẹyẹ pàtàkì bí ìgbéyàwó, àwọn àríyá, àti àwọn ìpàdé pàtàkì, láti fi hàn pé wọ́n bọ̀wọ̀ fún ayẹyẹ náà àti àwọn ènìyàn tó wà níbẹ̀. Ìtẹ́lọ́run tó wà nínú lílo wọn púpọ̀; àwọn tó bá wọ̀ wọ́n máa ń ní ìgbéraga àti ìgbẹ́kẹ̀lé ara ẹni, wọ́n sì nímọ̀lára pé àwọn ti wọ aṣọ tó dára, tó sì bọ́gbọ́n mu fún ayẹyẹ náà. Ó tún máa ń fún wọn ní ìmọ̀lára ìsopọ̀ pẹ̀lú àwọn baba ńlá wọn àti àṣà Yorùbá tó jinlẹ̀, èyí tó máa ń mú inú wọn dùn gidigidi.', 'Ó dára! Láti mú ìbéèrè yìí gbòòrò sí i, a lè fi àwọn ọ̀rọ̀ àti àwọn ìbéèrè mìíràn kún un tí ó jẹ mọ́ àṣà ìwọṣọ àwọn ọkùnrin Yorùbá, ìtàn rẹ̀, àti ipa rẹ̀ nínú àwùjọ.\n\nNíhìn-ín ni àbá kan:\n\n**Ìbéèrè Tí Ó Gbòòrò Sí I:**\n\n

In [73]:
response

{'question': 'Kí ló fà á tí kọmiṣanna náà fi ráyè ṣàbẹ̀wò sí odo Àṣà Sobi àti Agba, àti báwo ni àbẹ̀wò náà ṣe ṣe pàtàkì fún àwùjọ?',
 'answer': 'Àkọsílẹ̀ tí a fún mi kò ní ìsọfúnni kankan tó jọ mọ́ ìdí tí kọmisànná fi ṣàbẹ̀wò sí odo Àṣà Sobi àti Agba. Nítorí náà, kò ṣeé ṣe láti dáhùn ìbéèrè yìí nípa ìdí àbẹ̀wò náà tàbí bí ó ṣe ṣe pàtàkì fún àwùjọ láti inú àkọsílẹ̀ yìí. Kò sí àlàyé kankan tó tọ́ka sí irú ìṣẹ̀lẹ̀ bẹ́ẹ̀ nínú ọ̀rọ̀ tí a pèsè.',
 'verified': True,
 'context': 'nìyàn idàgbàsókè pàdé ìgbésè ìjọba gbé owó lé iye tí ǹ kan tí a lè ilé ayé yìí ò ni fọwọ́ si pé ó gúnrégé ọ̀rọ̀ ọjọ́sí ó jẹ́ ohun máa fi gbólóhùn wọn gbè ẹ̀kọ́ èdè yorùbá àti àwọn àwọn tó ṣe àgbékalẹ̀ nǹkan ìjọba wa sì sa ipá ìwúrí gbáà ló jẹ́ fún ẹni bá fẹ́ ṣe àṣeyege agbègbè láti ọjọ́ ketalelogun oṣù onímọ̀ ló gbà pé kía àwa yorùbá àti ìlẹ̀ nàìjíria nínú igbó níwọ̀n ìgbà tó wọ́n gbé jù sí gbàgede dúró lómìnira gẹ́gẹ́ bí i fún gbogbo ènìyàn nítorí kí agbára agbófìnró àpapọ̀ yìí pọ̀ ara ìyádúnní kò gbà á tọ́rọ̀ bá s

In [77]:
answer

'Àkọsílẹ̀ tí a pèsè kò ní àlàyé kankan nípa ìtúmọ̀ àṣà wíwọ̀ ṣòkòtò àti agbádá nínú àṣà Yorùbá. Kò sí ìsọfúnni kankan tó sọ nípa ìtẹ́lọ́run tàbí ìtumọ̀ àwọn aṣọ wọ̀nyí nínú àkọsílẹ̀ náà. Nítorí náà, kò ṣeé ṣe láti dáhùn ìbéèrè yìí láti inú àkọsílẹ̀ tí a fúnni.'

In [40]:
sampled_df.head()

,domain,doc_id,source,question_id,question,reference_answer,question_type,context_document,cluster
0,culture,culture_34,NaN,culture_34_q2,Kí ló fà á tí kọmiṣanna náà fi ráyè ṣàbẹ̀wò ...,Àbẹ̀wò kọmiṣanna náà jẹ́ pàtàkì nítorí ó fẹ́...,conceptual,kọmiṣanna fọrọ eto omi igbalode ṣabẹwo o...,0
1,culture,culture_32,Àsà àti ìse Yorùbá C. L. Adéoyé.docx,culture_32_q3,Kí ni ìtúmọ̀ àṣà wọ́ *ṣòkòtò àti agbádá* nínú ...,"Nínú àṣà Yorùbá, *ṣòkòtò àti agbádá* jẹ́ àmì ì...",cultural,"bádá a rnáa ni apá, ati àpò nigbá àyà ti ęn u ...",1
2,current affairs,current affairs_11,NaN,current affairs_11_q1,Kí ni ìtumọ̀ “kọ̀mpútà tàbí èrò oníṣirò” gẹ́gẹ...,Kọ̀mpútà tàbí èrò oníṣirò jẹ́ ẹ̀rọ onítànná tí...,factual,kọmpútà ns0ns revision parentid474043parent...,0
3,current affairs,current affairs_19,NaN,current affairs_19_q1,Kí ni US Open tàbí Open Amẹ́ríkà nípa ìtàn àti...,"Open Amẹ́ríkà, tí wọ́n tún mọ̀ sí United State...",factual,open amẹríkà ìdíje open tẹnìs amẹríka...,1
4,entertainment,entertainment_1,NaN,entertainment_1_q1,"Tálọ dá ẹgbẹ́ Ikorodu Broilers Hub sílẹ̀, àti ...",Ẹgbẹ́ Ikorodu Broilers Hub ni Ọmọ̀wé Olájídé B...,factual,a fẹẹ sọ iṣẹ agbẹ di okoowo to maa wun a...,0


In [42]:
ans=sampled_df["reference_answer"][0]
ans

'Àbẹ̀wò kọmiṣanna náà jẹ́ pàtàkì nítorí ó fẹ́ mọ bí iṣẹ́ ìtúnṣe omi igbalode ṣe ń lọ, àti láti rí i pé omi tó mọ́ wa fún mímu àti ìlò ojoojúmọ́. Ìrìnàjò náà tún jẹ́ ànfààní láti mọ àwọn ìpenija tó wà, pàápàá jùlọ nípa àjálù àrùn tó lè jẹ́yọ látàrí omi tó bàjẹ́. Nípa bẹ́ẹ̀, ó lè dá àwọn agbásẹ́ṣẹ́ lójú pé ìjọba ń bójú tó iṣẹ́ wọn. Ìṣe yìí fi hàn pé ìjọba ń gbìmọ̀ pẹ̀lú àwọn olùgbé láti dáàbò bo ìlera àti ìtẹ̀síwájú àwùjọ.'

In [ ]:
# ====================================
# 🚀 Evaluation Pipeline
# ====================================
all_results = []
start_all = time.time()

for model_name in model_names:
    print(f"\n🧠 Evaluating model: {model_name}")
    model_start = time.time()

    # Load model efficiently
    llm = set_model(model_name)
    free_memory()
    mod_rag =  modular_rag(dense_retriever=dense_retriever,sparse_retriever=sparse_retriever,embed_model=Settings.embed_model,verbose=True,llm=llm)

    for _, row in tqdm(sampled_df.iterrows(), total=len(sampled_df), desc=f"{model_name}"):
        domain = row["domain"]
        question = clean_text(row["question"])
        reference = clean_text(row["reference_answer"])
        context_doc = clean_text(row["context_document"])

        try:
            # 🔹 Generate answer
            response = mod_rag.query(question=question)
            answer = response["answer"]

            # 🔹 Metric computation
            context_score = evaluate_context_relevance(question, [context_doc])
            faith_score = evaluate_faithfulness(answer, [context_doc])
            relevance_score = evaluate_answer_relevance(question, answer)
            gen_metrics = evaluate_generation_metrics(question=question,
                contexts=[context_doc],
                answer=answer,
                reference=reference)

            composite_rag = round(
                (context_score + faith_score + relevance_score + gen_metrics["Composite_Gen"]) / 4, 3
            )
            composite_gen = gen_metrics.get("Composite_Gen", 0)
            composite_rag = round((context_score + faith_score + relevance_score + composite_gen) / 4, 3)
            print(relevance_score)
            print(context_score)
            print(relevance_score)
            print(gen_metrics)

            all_results.append({
                "model": model_name,
                "domain": domain,
                "question": question,
                "reference": reference,
                "generated_answer": answer,
                "context_relevance": context_score,
                "faithfulness": faith_score,
                "answer_relevance": relevance_score,
                **gen_metrics,
                "Composite_RAG_Score": composite_rag
            })

        except Exception as e:
            print(f"[WARN] Skipped item: {e}")
            continue

    print(f"✅ Completed {model_name} in {round(time.time() - model_start, 2)} sec")
    free_memory()

# ====================================
# 🧾 Aggregate Results
# ====================================
df_all = pd.DataFrame(all_results)

if df_all.empty:
    print("\n⚠️ No evaluation results were recorded — check generation or metric functions.")
else:
    print("\n📊 Sample of Evaluation Results:")
    print(df_all.head())

    # Model-wise summary (average metrics)
    summary = (
        df_all.groupby("model")[["context_relevance", "faithfulness", "Composite_RAG_Score"]]
        .mean()
        .sort_values("Composite_RAG_Score", ascending=False)
    )

    print("\n📈 Yoruba RAG Evaluation Summary:")
    print(summary)

print(f"\n⏱️ Total Evaluation Time: {round(time.time() - start_all, 2)} sec")


🧠 Evaluating model: mistralai/Mistral-7B-Instruct-v0.3
🧠 Detected free GPU memory: 0.4 GB
🔄 Loading mistralai/Mistral-7B-Instruct-v0.3...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

🎯 Filtered: 2 / 7 nodes remain
🔄 Reranked 2 nodes | Top scores: ['0.913', '0.791']


mistralai/Mistral-7B-Instruct-v0.3:  10%|█         | 1/10 [00:25<03:45, 25.08s/it]

[WARN] Skipped item: CUDA out of memory. Tried to allocate 52.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 36.12 MiB is free. Process 10710 has 14.70 GiB memory in use. Of the allocated memory 3.40 GiB is allocated by PyTorch, and 112.74 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

🔍 Query: Kí ni ìtúmọ̀ àṣà wọ́ *ṣòkòtò àti agbádá* nínú àṣà Yorùbá, àti kí ni ìtẹ́lọ́run tó wà nínú lílo wọn?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🎯 Filtered: 1 / 7 nodes remain
🔄 Reranked 1 nodes | Top scores: ['0.915']


In [ ]:
df_all